# Taller 3: Implementación de Relé Virtual con módulo de adquisición de datos y DSP para protección de Línea de Transmisión mediante la técnica de Transformada Wavelet

Integrantes: 
- Alejandra Garzón
- Danilo Dorado
- Ismael Montoya
- Sergio Turizo

**Objetivo**
- Aplicar los conceptos fundamentales relacionados con las fases de adquisición de datos, procesamiento básico digital de señales de tensión/corriente, y cálculo básico de fasores para aplicaciones de relés de protección. (Entregar carpeta con reporte del diseño e implementación de cada módulo y archivos de simulación).

**En este documento encontrarás las fases de adquisición de datos y procesamiento básico **
- Etapa de adquisición:
    - Sample & Hold
    - Analog to Digital Conversion
    
- Etapa de procesamiento básico:
    - Filtrado digital
    - Transformada discreta de Fourier (DFT)
    - Ventaneo móvil
    


## 1 - Librerías ##

First, let's run the cell below to import all the packages that you will need during this assignment. 
- [numpy](www.numpy.org) is the fundamental package for scientific computing with Python.
- [matplotlib](http://matplotlib.org) is a famous library to plot graphs in Python.
- [Slider](https://pypi.org/project/window-slider) is a independent module with windowing functions included


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
import numpy as np
from scipy import signal
import scipy.io as sio
from window_slider import Slider
import cmath
import os
import pywt
from tkinter import *
%matplotlib tk

In [ ]:
def select_event(num):
    ev_folder = 'Eventos_T3/Evento_'+str(num)
    matfiles = os.listdir(ev_folder) # lista de mediciones del evento
    return matfiles, ev_folder+"/"
f = open("Eventos_T3/Lista_eventos.txt", "r")
print(f.read())
number =  input("¿Cuál es el evento que desea evaluar? ")
mat_files, ev_f = select_event(int(number))
#print(mat_files, ev_f)

## 2 - Etapa de Adquisición - Archivos .MAT ##

**Implementación**: Dado la librería scikit se procede a importar un objeto de tipo .mat para su lectura y posterior almacenamiento de la siguiente forma:
   - Cada señal adquirida se convertirá en un arreglo numérico y será almacenada en una lista
   - La estampa de tiempo será independiente e igual para todas las señales.
   - Los rotulos/ leyendas de cada señal serán las que se obtuvieron en el archivo comtrade importado.

Se presenta las gráficas de estas señales relacionadas con sus variables: voltaje y corriente.


In [ ]:
matCT1 = sio.loadmat(ev_f+mat_files[0])
t, iCt1a, iCt1b, iCt1c = matCT1['t'], matCT1['iCt1a'], matCT1['iCt1b'], matCT1['iCt1c']
matVT1 = sio.loadmat(ev_f+mat_files[3])
t, vVt1a, vVt1b, vVt1c = matVT1['t'], matVT1['vVt1a'], matVT1['vVt1b'], matVT1['vVt1c']
matCT2 = sio.loadmat(ev_f+mat_files[1])
t, iCt2a, iCt2b, iCt2c = matCT2['t'], matCT2['iCt2a'], matCT2['iCt2b'], matCT2['iCt2c']
matVT2 = sio.loadmat(ev_f+mat_files[4])
t, vVt2a, vVt2b, vVt2c = matVT2['t'], matVT2['vVt2a'], matVT2['vVt2b'], matVT2['vVt2c']
matCT3 = sio.loadmat(ev_f+mat_files[2])
t, iCt3a, iCt3b, iCt3c = matCT3['t'], matCT3['iCt3a'], matCT3['iCt3b'], matCT3['iCt3c']
matVT3 = sio.loadmat(ev_f+mat_files[5])
t, vVt3a, vVt3b, vVt3c = matVT3['t'], matVT3['vVt3a'], matVT3['vVt3b'], matVT3['vVt3c']

signalsCT1 = [iCt1a, iCt1b, iCt1c]
signalsVT1 = [vVt1a, vVt1b, vVt1c]
signalsCT2 = [iCt2a, iCt2b, iCt2c]
signalsVT2 = [vVt2a, vVt2b, vVt2c]
signalsCT3 = [iCt3a, iCt3b, iCt3c]
signalsVT3 = [vVt3a, vVt3b, vVt3c]
    
plt.figure()
plt.subplot(2,1,1)
plt.plot(t, iCt1a)
plt.xlabel("Tiempo (s)")
plt.ylabel("Corriente (A)")
plt.title("Mediciones CT1")
plt.plot(t, iCt1b)
plt.plot(t, iCt1c)
plt.legend(['iCt1a', 'iCt1b', 'iCt1c'])
plt.grid()
plt.subplot(2,1,2)
plt.plot(t, vVt1a)
plt.plot(t, vVt1b)
plt.plot(t, vVt1c)
plt.xlabel("Tiempo (s)")
plt.ylabel("Voltaje (V)")
plt.legend(['vVt1a', 'vVt1b', 'vVt1c'])
plt.grid()
plt.show()

plt.figure()
plt.subplot(2,1,1)
plt.plot(t, iCt2a)
plt.plot(t, iCt2b)
plt.plot(t, iCt2c)
plt.title("Mediciones CT2")
plt.xlabel("Tiempo (s)")
plt.ylabel("Corriente (A)")
plt.legend(['iCt2a', 'iCt2b', 'iCt2c'])
plt.grid()
plt.subplot(2,1,2)
plt.plot(t, vVt2a)
plt.plot(t, vVt2b)
plt.plot(t, vVt2c)
plt.legend(['vVt2a', 'vVt2b', 'vVt2c'])
plt.xlabel("Tiempo (s)")
plt.ylabel("Voltaje (V)")
plt.grid()
plt.show()

plt.figure()
plt.subplot(2,1,1)
plt.plot(t, iCt3a)
plt.plot(t, iCt3b)
plt.plot(t, iCt3c)
plt.title("Mediciones CT3")
plt.xlabel("Tiempo (s)")
plt.ylabel("Corriente (A)")
plt.legend(['iCt3a', 'iCt3b', 'iCt3c'])
plt.grid()
plt.subplot(2,1,2)
plt.plot(t, vVt3a)
plt.plot(t, vVt3b)
plt.plot(t, vVt3c)
plt.legend(['vVt3a', 'vVt3b', 'vVt3c'])
plt.xlabel("Tiempo (s)")
plt.ylabel("Voltaje (V)")
plt.grid()
plt.show()
print(len(iCt1a))

**# Señales importadas esperadas**: 
<table style="width:15%">
  <tr>
    <td>Voltaje</td>
    <td> 3 </td> 
  </tr>
  
  <tr>
    <td>Corriente</td>
    <td> 3 </td> 
  </tr>
    
</table>


## 3 - Sample & Hold y Codificación binaria ##

En esta etapa se procede a implementar dos funciones que permiten realizar estos procesos de manera independiente aunque son secuenciales. De esta manera, con respecto a la lista de señales de la etapa anterior, se procede a muestrear y cuantizar cada señal

### 3.1 - Sample & Hold

**Desarrollo**: Teniendo la frecuencia de muestreo cómo parametro de entrada, se implementa la función para realizar el muestreo

In [ ]:
def samp_hold(sig, t, fs):

    ts=1/60 # ts de la señal de entrada
    sample_period=ts/fs
    sample_time=np.arange(0, t[len(t)-1], sample_period)
    sample_sig = np.empty(len(sample_time))
    sample_sig=signal.resample(sig, len(sample_time), t = t)
    return sample_sig[0], sample_time

### 3.2 - Quantization

**Desarrollo:** Se aplica con un nivel de cuantización de 8 bits el proceso con la señal muestreada.

In [ ]:
def quantize(sampled_sig, sampled_t):
    nbits = 8 # 256 quantization levels
    qLevels = 2**nbits
    signalMin = np.min(sampled_sig)
    signalMax = np.max(sampled_sig)
    scalingFactor = np.divide((signalMax-signalMin),(qLevels))
    qsignal = np.round(sampled_sig / scalingFactor) * scalingFactor
    #plt.figure()
    #plt.plot(sampled_t, sampled_sig, 'r', sampled_t, qsignal, '--k')
    #plt.xlabel("Time")
    #plt.ylabel("Amplitude")
    #plt.grid(True)
    return qsignal

### 3.3 - Implementación: Etapa de adquisición

**Desarrollo:** Dadas las funciones definidas en 3.1 y 3.2 se procede a emplearlas en las señales obtenidas en la sección 2 

In [ ]:
signalsCT1_stage2 = list() # Almacenamiento de las señales muestreadas
signalsVT1_stage2 = list() # Almacenamiento de las señales muestreadas
signalsCT2_stage2 = list() # Almacenamiento de las señales muestreadas
signalsVT2_stage2 = list() # Almacenamiento de las señales muestreadas
signalsCT3_stage2 = list() # Almacenamiento de las señales muestreadas
signalsVT3_stage2 = list() # Almacenamiento de las señales muestreadas

fs =  input("¿Cuál es la frecuencia de muestreo del relé? ")
nCT = 1
nVT = 1
for sigc1 in signalsCT1:
    values_C, time = samp_hold(sigc1, t, float(fs))
    qvalues_C = quantize(values_C, time)
    signalsCT1_stage2.append(np.dot(nCT,qvalues_C).reshape((len(qvalues_C),)))
    

for sigv1 in signalsVT1:
    values_V, time = samp_hold(sigv1, t, float(fs))
    qvalues_V = quantize(values_V, time)
    signalsVT1_stage2.append(np.dot(nVT,qvalues_V).reshape((len(qvalues_C),)))
    

for sigc2 in signalsCT2:
    values_C, time = samp_hold(sigc2, t, float(fs))
    qvalues_C = quantize(values_C, time)
    signalsCT2_stage2.append(np.dot(nCT,qvalues_C).reshape((len(qvalues_C),)))
    

for sigv2 in signalsVT2:
    values_V, time = samp_hold(sigv2, t, float(fs))
    qvalues_V = quantize(values_V, time)
    signalsVT2_stage2.append(np.dot(nVT,qvalues_V).reshape((len(qvalues_C),)))
    
    
for sigc3 in signalsCT3:
    values_C, time = samp_hold(sigc3, t, float(fs))
    qvalues_C = quantize(values_C, time)
    signalsCT3_stage2.append(np.dot(nCT,qvalues_C).reshape((len(qvalues_C),)))
    

for sigv3 in signalsVT3:
    values_V, time = samp_hold(sigv3, t, float(fs))
    qvalues_V = quantize(values_V, time)
    signalsVT3_stage2.append(np.dot(nVT,qvalues_V).reshape((len(qvalues_C),)))
    

## 4 - Etapa de Procesamiento Wavelet ## 

En esta etapa tenemos los siguientes pasos para su implementación:
1. Definir el número de niveles de energía (3)
2. Definir la wavelet madre que se empleará: Daubechies 4th order
3. Recorrer las señales mediante ventanas de tamaño constante:
    - Utilizar la señal proveniente de la etapa anterior (señal cuantizada)
    - Calcular la transformada de wavelet discreta
    - Guardar cada desplazamiento de la ventana en una lista

Después de estos pasos se procederá a la implementación de la función diferencial


### 4.1 - Ventaneo dinámico y DWT

**Desarrollo:** Con base en la creación de un objeto ventana con un búffer de longitud igual a la frecuencia de muestreo (# de muestras por ciclo) se realiza el ventaneo dinámico a la señal codificada.



In [ ]:
def movil_windowing(fs, quan_sig):
    buffer_len = fs
    overlap_count = fs-1 #desplazamiento por ventana
    slider = Slider(buffer_len, overlap_count)
    slider.fit(quan_sig)
    output=list()
    while True:
        window_data = slider.slide()
        E_icum = np.sum(np.power(window_data, 2))
        output.append(E_icum)
        #print(window_data)
        
        if slider.reached_end_of_list(): break
    return output

def energy_windowing(fs, I_op, I_res ):   
    N_tot = len(I_op)-fs
    Xc = [0]*N_tot

    # Ciclo para el ventaneo
    for i in np.arange(N_tot):
        if I_op[i] > 0.45*I_res[i]:
            xi = I_op[i:i+fs]
            Eop_sum = 0
            for k in np.arange(fs):
                Xc_temp=xi[k]**2
                Eop_sum=Eop_sum+Xc_temp
            Xc[i] = Eop_sum
        else:
            xi = I_op[i:i+fs]
            Eop_sum2 = 0
            for k in np.arange(fs):
                Xc_temp2 = xi[k]
                Eop_sum2 += Xc_temp2
            Xc[i] = (Eop_sum2/fs)**2
    return Xc

In [ ]:
signals_CT1_dwt = list()
signals_CT2_dwt = list()
signals_CT3_dwt = list()
for current in signalsCT1_stage2:
    coeffs = pywt.wavedec(current, 'db4', level=3)
    dwt_approx = coeffs[0]
    signals_CT1_dwt.append(dwt_approx)
for current2 in signalsCT2_stage2:
    coeffs = pywt.wavedec(current2, 'db4', level=3)
    dwt_approx = coeffs[0]
    signals_CT2_dwt.append(dwt_approx)
for current3 in signalsCT3_stage2:
    coeffs = pywt.wavedec(current3, 'db4', level=3)
    dwt_approx = coeffs[0]
    signals_CT3_dwt.append(dwt_approx)

## 5. Implementación de la protección diferencial ##

**Desarrollo:** Con base en la aproximación de nivel 3 de la transformada Wavelet de las corrientes de fase de cada transformador de corriente se procede a estimar la corriente de operación y restricción



In [ ]:
IdifA = signals_CT1_dwt[0]+signals_CT2_dwt[0]+signals_CT3_dwt[0]
IdifB = signals_CT1_dwt[1]+signals_CT2_dwt[1]+signals_CT3_dwt[1]
IdifC = signals_CT1_dwt[2]+signals_CT2_dwt[2]+signals_CT3_dwt[2]
IavA = (np.abs(signals_CT1_dwt[0])+np.abs(signals_CT2_dwt[0])+np.abs(signals_CT3_dwt[0]))/3
IavB = (np.abs(signals_CT1_dwt[1])+np.abs(signals_CT2_dwt[1])+np.abs(signals_CT3_dwt[1]))/3
IavC = (np.abs(signals_CT1_dwt[2])+np.abs(signals_CT2_dwt[2])+np.abs(signals_CT3_dwt[2]))/3
print(len(IdifA))

In [ ]:
ts = np.linspace(0,0.3, len(IdifA))
plt.figure()
plt.subplot(3,1,1)
plt.plot(ts, IdifA, color='b')
plt.plot(ts, IavA, color='k')
plt.title('Aprox DWT de I-operación vs I-restricción')
plt.ylabel('Fase A')
plt.legend(['I-ope', 'I-res'], loc='upper right')
plt.grid()
plt.subplot(3,1,2)
plt.plot(ts, IdifB, color='r')
plt.plot(ts, IavB, color='k')
plt.ylabel('Fase B')
plt.grid()
plt.legend(['I-ope', 'I-res'], loc='upper right')
plt.subplot(3,1,3)
plt.plot(ts, IdifC, color='g')
plt.plot(ts, IavC, color='k')
plt.ylabel('Fase C')
plt.legend(['I-ope', 'I-res'], loc='upper right')
plt.xlabel('Tiempo (s)')
plt.grid()
plt.show()

### 5.1 - Estimación del nivel de energía por fase

**Desarrollo:** A partir de las corrientes de operación y restricción estimadas se procede a estimar la energía para visualizar las fases falladas de acuerdo al siguiente criterio de disparo -
E_op>E_res

Asimismo, el nivel de energía se cálcula mediante el ventaneo dinámico de las corrientes obtenidas en la celda anterior. La forma que se emplea es la suma de los cuadrados de cada valor dentro de una ventana.


In [ ]:
E_opA2 = energy_windowing(int(fs), IdifA, IavA)
E_opB2 = energy_windowing(int(fs), IdifB, IavB)
E_opC2 = energy_windowing(int(fs), IdifC, IavC)
E_resA2 = movil_windowing(int(fs), IavA)
E_resB2= movil_windowing(int(fs), IavB)
E_resC2 = movil_windowing(int(fs), IavC)
print(len(E_opA2), len(E_resB2))
plt.figure()
plt.subplot(3,1,1)
plt.plot(E_opA2, color='b')
plt.plot(E_resA2, color='k')
plt.title('Nivel de energía de I-operación vs I-restricción')
plt.ylabel('Fase A')
plt.legend(['E-ope', 'E-res'], loc='upper left')
plt.grid()
plt.subplot(3,1,2)
plt.plot(E_opB2, color='r')
plt.plot(E_resB2, color='k')
plt.ylabel('Fase B')
plt.legend(['E-ope', 'E-res'], loc='upper left')
plt.grid()
plt.subplot(3,1,3)
plt.plot(E_opC2, color='g')
plt.plot(E_resC2, color='k')
plt.ylabel('Fase C')
plt.xlabel('Samples')
plt.legend(['E-ope', 'E-res'], loc='upper left')
plt.grid()
plt.show()

### 5.2 - Identificación y clasificación de la falla

**Desarrollo:** Se desarrolla la lógica de detección y disparo de acuerdo con los valores estimados y definiendo el factor K que se emplea en esta función. La primera salida será las señales de detección y disparo del evento por cada fase.


In [ ]:
factor = int(input("Ingrese el factor (%): "))/100
t_ev = np.linspace(0,0.3, len(IdifA))
disparo_a = 0
trigger_A = np.zeros((len(IdifA),))
posA = 0 
for i in range(len(IdifA)):
    if IdifA[i]>IavA[i]*factor:
        disparo_a = 1
        if posA == 0:
            posA = i
            trigger_A[i] = 1
        #print(disparo_a,i)

disparo_b = 0
trigger_B = np.zeros((len(IdifB),))
posB = 0
for i in range(len(IdifB)):
    if IdifB[i]>IavB[i]*factor:
        disparo_b = 1
        if posB == 0:
            posB = i
            trigger_B[i] = 1
        #print(disparo_b,i)

disparo_c = 0
trigger_C = np.zeros((len(IdifC),))
posC = 0
for i in range(len(IdifC)):
    if IdifC[i]>IavC[i]*factor:
        disparo_c = 1
        if posC == 0:
            posC = i
            trigger_C[i] = 1
        #print(disparo_c,i)

disparo = 0

a = ''
# Identificación - de una falla
if (disparo_a or disparo_b or disparo_c)==1:
    disparo = 1
    elapsed = np.max([t_ev[posA], t_ev[posB], t_ev[posC]])
    trip = elapsed + (1/120)
else: 
    disparo = 0
    elapsed = 0
    trip = 0
    a ='Falla externa'

# Identificación del tipo de - Falla trifásica, monofásica o bifásica
if (disparo_c and disparo_b and disparo_a) == 1:
    a = 'Falla trifásica'
elif ((disparo_a ==1 ) and (disparo_b == 0) and (disparo_c == 0)) or ((disparo_a ==0 ) and (disparo_b == 1) and (disparo_c == 0)) or((disparo_a ==0 ) and (disparo_b == 0) and (disparo_c == 1)):
    if disparo_a == 1: 
        a = 'Falla monofásica fase a'
    elif disparo_b == 1:
        a = 'Falla monofásica fase b'
    elif disparo_c == 1:
        a = 'Falla monofásica fase c'
    
elif ((disparo_a ==1 ) and (disparo_b == 1) and (disparo_c == 0)) or ((disparo_a ==0 ) and (disparo_b == 1) and (disparo_c == 1)) or((disparo_a ==1 ) and (disparo_b == 0) and (disparo_c == 1)):
    if (disparo_a and disparo_b) == 1:
        a = 'Falla bifásica ab'
    elif(disparo_a and disparo_c) == 1:  
        a = 'Falla bifásica ac'
        
    elif(disparo_b and disparo_c) == 1:
        a = 'Falla bifásica bc'

print(a)
print(disparo)
print(elapsed)
print(trip)

plt.figure()
plt.subplot(3,1,1)
plt.plot(t_ev,trigger_A)
plt.plot(t_ev+[1/120]*len(t_ev),trigger_A)
plt.title('Señales digitales del evento')
plt.ylabel('Fase A')
plt.legend(['Detección', 'Disparo'])
plt.grid()
plt.subplot(3,1,2)
plt.plot(t_ev,trigger_B)
plt.plot(t_ev+[1/120]*len(t_ev),trigger_B)
plt.legend(['Detección', 'Disparo'])
plt.ylabel('Fase B')
plt.grid()
plt.subplot(3,1,3)
plt.plot(t_ev,trigger_C)
plt.plot(t_ev+[1/120]*len(t_ev),trigger_C)
plt.legend(['Detección', 'Disparo'])
plt.ylabel('Fase C')
plt.xlabel('Tiempos (s)')
plt.grid()
plt.show()

### 5.3 - Reporte Final del Evento

**Desarrollo:** Cuadro de dialogo que presenta la información relevante al tipo de falla, fases involucradas, tiempo y orden lógica de detección así como de disparo.


In [ ]:
window = Tk()
window.geometry("400x100+400+100")
window.title("Alerta de falla")  

label_tipo_de_falla = Label(window,text = "Tipo de falla: ")
label_tipo_de_falla.grid(row = 0,column = 0, sticky = N)

t1 = Text(window,height = 1, width = 30)
t1.grid(row = 0,column = 1, sticky = N)

label_senal_de_disparo = Label(window,text = "Señal de disparo: ")
label_senal_de_disparo.grid(row = 1,column = 0, sticky = N)

t2 = Text(window,height = 1, width = 30)
t2.grid(row = 1,column = 1, sticky = N)

label_tiempo_de_deteccion = Label(window,text = "Tiempo de detección de falla: ")
label_tiempo_de_deteccion.grid(row = 2,column = 0, sticky = N)

t3 = Text(window,height = 1, width = 30)
t3.grid(row = 2,column = 1, sticky = N)

label_tiempo_de_disparo = Label(window,text = "Tiempo de disparo: ")
label_tiempo_de_disparo.grid(row = 3,column = 0, sticky = N)

t4 = Text(window,height = 1, width = 30)
t4.grid(row = 3,column = 1, sticky = N)

t1.insert(END,a)
t2.insert(END,disparo)
t3.insert(END,np.round(elapsed, 3))
t4.insert(END,np.round(trip,3))

window.mainloop()